# **💁🏻🗨️💁🏻‍♂️대화 요약 Code**
> 해당 대회는 **Upstage AI Lab** 과정에서 비공개로 진행된 내부 대회이며 **일상 대화에 대한 요약**을 효과적으로 생성하는 모델을 개발하는 대회입니다.  
> 해당 대회에서 주어진 데이터셋은 영어 일상 대화 요약 Task에서 많이 활용되는 **[DialogSum](https://huggingface.co/datasets/knkarthick/dialogsum)** 데이터셋을 **한국어로 번역한** 데이터라는 점이 대회의 특징입니다.  
> 해당 코드는 **T5 모델**을 사용하여 DialogSum 데이터 셋으로 FineTuning을 진행합니다. 또한 성능은 떨어지지만 선택적으로 **GPT로 합성한 데이터**를 사용해 볼 수 있습니다.

## 1 Prepare in advance

In [1]:
import pandas as pd
import os
import re
import json
import yaml
import numpy as np
from glob import glob
from tqdm import tqdm
from pprint import pprint
import torch
import pytorch_lightning as pl
from rouge import Rouge # 모델의 성능을 평가하기 위한 라이브러리입니다.
# from konlpy.tag import Mecab

from torch.utils.data import Dataset , DataLoader
from transformers import AutoTokenizer
from transformers import AutoModelForSeq2SeqLM, T5Config
from transformers import T5Tokenizer, T5ForConditionalGeneration
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer
from transformers import Trainer, TrainingArguments
from transformers import EarlyStoppingCallback

import wandb # 모델 학습 과정을 손쉽게 Tracking하고, 시각화할 수 있는 라이브러리입니다.
import random

ModuleNotFoundError: No module named 'rouge'

In [ ]:
# 데이터 EDA용 코드
def print_data(df, start=0, end=None, count=5, print_random=False, mode='loc'):
    if print_random:
        samples_idx = df.sample(count).index
    else:
        if mode == 'loc':
            if end==None:
                samples_idx = df.loc[start:start+count].index
            else:
                samples_idx = df.loc[start:end].index
        elif mode == 'iloc':
            if end==None:
                samples_idx = df.iloc[start:start+count].index
            else:
                samples_idx = df.iloc[start:end].index
                
    print(samples_idx)
    for i in samples_idx:
        fname = df.loc[i, 'fname'] if 'fname' in df.columns else None
        dialogue = df.loc[i, 'dialogue'] if 'dialogue' in df.columns else None
        summary = df.loc[i, 'summary'] if 'summary' in df.columns else None
        pred = df.loc[i, 'pred'] if 'pred' in df.columns else None
        
        print("="*50)
        print(f"[{fname}]")
        if dialogue:
            print("[Dialogue]")
            print(dialogue)
        if summary:
            print("[Summary]")
            print(summary)
        if pred:
            print("[Prediction]")
            print(pred)
        
        if 'rouge_1' in df.columns:
            rouge_1 = df.loc[i, 'rouge_1']
            rouge_2 = df.loc[i, 'rouge_2']
            rouge_L = df.loc[i, 'rouge_L']
            print(f"rouge_1: {rouge_1:.4f}, rouge_2: {rouge_2:.4f}, rouge_L: {rouge_L:.4f}")

### 1.1 Config file 만들기
- 모델 생성에 필요한 다양한 매개변수 정보를 저장할 수 있습니다.  
  따라서, 코드 상에서 모델의 매개변수를 설정할 수도 있지만 독립적인 매개변수 정보 파일을 생성하여 관리할 수 있습니다.

In [ ]:
# config 설정에 tokenizer 모듈이 사용되므로 미리 tokenizer를 정의해줍니다.
tokenizer = AutoTokenizer.from_pretrained("eenzeenee/t5-base-korean-summarization")

In [ ]:
config_data = {
    "general": {
        "data_path": "../data/", # 모델 생성에 필요한 데이터 경로를 사용자 환경에 맞게 지정합니다.
        "model_name": "eenzeenee/t5-base-korean-summarization", # 불러올 모델의 이름을 사용자 환경에 맞게 지정할 수 있습니다.
        "output_dir": "../model" # 모델의 최종 출력 값을 저장할 경로를 설정합니다.
    },
    "tokenizer": {
        "encoder_max_len": 768,
        "decoder_max_len": 128,
        "bos_token": f"{tokenizer.bos_token}",
        "eos_token": f"{tokenizer.eos_token}",
        # 특정 단어들이 분해되어 tokenization이 수행되지 않도록 special_tokens을 지정해줍니다.
        "special_tokens": ['#Person1#', '#Person2#', '#Person3#', '#Person4#', '#Person5#', '#Person6#', '#Person7#', '#CarNumber#', '#DateOfBirth#', '#CardNumber#', '#SSN#', '#Email#', '#Address#', '#PhoneNumber#', '#PassportNumber#']
    },
    "training": {
        "overwrite_output_dir": True,
        "num_train_epochs": 20,
        "learning_rate": 2e-5,
        "per_device_train_batch_size": 8,
        "per_device_eval_batch_size": 8,
        "warmup_ratio": 0.1,
        "weight_decay": 0.01,
        "lr_scheduler_type": 'cosine',
        "optim": 'adamw_torch',
        "gradient_accumulation_steps": 1,
        "evaluation_strategy": 'epoch',
        "save_strategy": 'epoch',
        "save_total_limit": 5,
        "fp16": False,
        "fp16_full_eval": False,
        "load_best_model_at_end": True,
        "seed": 42,
        "logging_dir": "./logs",
        "logging_strategy": "epoch",
        "predict_with_generate": True,
        "generation_max_length": 128,
        "do_train": True,
        "do_eval": True,
        "early_stopping_patience": 3,
        "early_stopping_threshold": 0.0001,
        "report_to": "wandb" # (선택) wandb를 사용할 때 설정합니다.
    },
    # Wandb를 사용하기 위해서는 아래 변수들 설정
    "wandb": {
        "entity": "",
        "project": "",
        "name": ""
    },
    "inference": {
        "ckt_path": "model ckt path", # 사전 학습이 진행된 모델의 checkpoint를 저장할 경로를 설정합니다.
        "result_path": "./prediction/",
        "no_repeat_ngram_size": 6,
        "early_stopping": True,
        "generate_max_length": 128,
        "num_beams": 6,
        "batch_size" : 8,
        # 정확한 모델 평가를 위해 제거할 불필요한 생성 토큰들을 정의합니다.
        "remove_tokens": ['<usr>', f"{tokenizer.bos_token}", f"{tokenizer.eos_token}", f"{tokenizer.pad_token}"]
    }
}

In [ ]:
# 모델의 구성 정보를 YAML 파일로 저장
config_path = "./config.yaml"
with open(config_path, "w") as file:
    yaml.dump(config_data, file, allow_unicode=True)

### 1.2 Configuration 불러오기

In [ ]:
# 저장된 config 파일을 로드
config_path = "./config.yaml"

with open(config_path, "r") as file:
    loaded_config = yaml.safe_load(file)

pprint(loaded_config)

{'general': {'data_path': '../data/',
             'model_name': 'eenzeenee/t5-base-korean-summarization',
             'output_dir': '../model'},
 'inference': {'batch_size': 8,
               'ckt_path': 'model ckt path',
               'early_stopping': True,
               'generate_max_length': 128,
               'no_repeat_ngram_size': 6,
               'num_beams': 6,
               'remove_tokens': ['<usr>', 'None', '</s>', '<pad>'],
               'result_path': './prediction/'},
 'tokenizer': {'bos_token': 'None',
               'decoder_max_len': 128,
               'encoder_max_len': 768,
               'eos_token': '</s>',
               'special_tokens': ['#Person1#',
                                  '#Person2#',
                                  '#Person3#',
                                  '#Person4#',
                                  '#Person5#',
                                  '#Person6#',
                                  '#Person7#',
                          

In [ ]:
# # 사용자가 사용할 wandb config 설정
# loaded_config['wandb']['entity'] = ""
# loaded_config['wandb']['name'] = ""
# loaded_config['wandb']['project'] = ""

### 1.3 데이터 불러와서 확인해보기

In [ ]:
# config에 저장된 데이터 경로를 통해 train과 validation data를 불러옵니다.
data_path = loaded_config['general']['data_path']

# train data의 구조와 내용을 확인합니다.
train_df = pd.read_csv(os.path.join(data_path,'train.csv'))
train_df.tail(1)

,fname,dialogue,summary,topic
12456,train_12459,"#Person1#: 엄마, 다음 토요일에 이 삼촌네 가족을 방문하기 위해 비행기를 ...",#Person1#은 다음 토요일에 이 삼촌네를 방문할 때 가방을 어떻게 싸야 할지 ...,짐 싸기


In [ ]:
# validation data의 구조와 내용을 확인합니다.
val_df = pd.read_csv(os.path.join(data_path,'dev.csv'))
val_df.tail(1)

,fname,dialogue,summary,topic
498,dev_499,#Person1#: 여름이 다 되어간다는 게 믿기지 않아.\r\n#Person2#:...,#Person2#는 #Person1#에게 여름 휴가 동안 파티를 도와주는 회사에서 ...,여름 휴가


## 2 Data Process

In [ ]:
# 정규식 패턴에 해당하는 문자열을 replace하는 함수
def remove_extra_spc(x, pattern, replace_text):
    return re.sub(pattern, replace_text, x)

In [ ]:
# 데이터 전처리
def train_process(df):
    df['dialogue'] = df['dialogue'].str.strip()
    df['summary'] = df['summary'].str.strip()

    pattern = r"\r"
    df['dialogue'] = df['dialogue'].apply(lambda x: remove_extra_spc(x, pattern, ""))

    df['dialogue_list'] = df['dialogue'].str.split("\n")
    
    # pattern = r"[^a-zA-Z가-힣ㄱ-ㅎㅏ-ㅣ0-9\s!#$%&*()+-=~‘’'\":.,/?…—–\x08><]"
    # df['dialogue'] = df['dialogue'].apply(lambda x: remove_extra_spc(x, pattern, ""))

    # pattern = r"[>‘’]"
    # df['dialogue'] = df['dialogue'].apply(lambda x: remove_extra_spc(x, pattern, "\'"))

    # pattern = r"…"
    # df['dialogue'] = df['dialogue'].apply(lambda x: remove_extra_spc(x, pattern, "..."))

    pattern = r"[—–\x08]"
    df['dialogue'] = df['dialogue'].apply(lambda x: remove_extra_spc(x, pattern, " "))

    pattern = r"ㅇ"
    df['dialogue'] = df['dialogue'].apply(lambda x: remove_extra_spc(x, pattern, "으"))

    pattern = r"[ㄱ-ㅊㅌ-ㅎ]"
    df['dialogue'] = df['dialogue'].apply(lambda x: remove_extra_spc(x, pattern, ""))

    pattern = r"##"
    df['dialogue'] = df['dialogue'].apply(lambda x: remove_extra_spc(x, pattern, "#"))

    pattern = r"#Person#"
    df['dialogue'] = df['dialogue'].apply(lambda x: remove_extra_spc(x, pattern, "#Person1#"))

    df.loc[839, 'dialogue'] = df.loc[839, 'dialogue'].replace("사람1", "Person1#: ")

    pattern = r"사람1#"
    df['dialogue'] = df['dialogue'].apply(lambda x: remove_extra_spc(x, pattern, "#Person1#"))

    pattern = r"^\"#Person"
    df['dialogue'] = df['dialogue'].apply(lambda x: remove_extra_spc(x, pattern, "#Person"))

    pattern = r"\"$"
    df['dialogue'] = df['dialogue'].apply(lambda x: remove_extra_spc(x, pattern, ""))

    pattern = r"#Person 2#"
    df['dialogue'] = df['dialogue'].apply(lambda x: remove_extra_spc(x, pattern, "#Person2#"))

    df.loc[7505, 'dialogue'] = df.loc[7505, 'dialogue'].replace("#Person2# 좋은 아침입니다.", "#Person2#: 좋은 아침입니다.")

    df.loc[9547, 'dialogue'] = df.loc[9547, 'dialogue'].replace("#Person1: 성인을 대상으로", "#Person1#: 성인을 대상으로")
    df.loc[9547, 'dialogue'] = df.loc[9547, 'dialogue'].replace("#Person2: 제 첫 두 소설은", "#Person2#: 제 첫 두 소설은")

    df.loc[9548, 'dialogue'] = df.loc[9548, 'dialogue'].replace("#Person2: 분명히 그럴 거에요", "#Person2#: 분명히 그럴 거에요")
    df.loc[9548, 'dialogue'] = df.loc[9548, 'dialogue'].replace("#Person1: 우리는 좋은 인상을", "#Person1#: 우리는 좋은 인상을")

    df.loc[9750, 'dialogue'] = df.loc[9750, 'dialogue'].replace("Person1#: 이번 여름에 당신의", "#Person1#: 이번 여름에 당신의")

    df.loc[9779, 'dialogue'] = df.loc[9779, 'dialogue'].replace("Person1#: 우리 오늘 운이 좋네.", "#Person1#: 우리 오늘 운이 좋네.")

    # 각줄의 화자의 표시가 이상하게 돼 있는 데이터 수정
    df.loc[969, 'dialogue'] = df.loc[969, 'dialogue'].replace("제프, 이 광고를 봐!", "#Person1#: 제프, 이 광고를 봐!")
    df.loc[1213, 'dialogue'] = df.loc[1213, 'dialogue'].replace("#하지만 장기간의 무중력 상태,", "#Person1#: 하지만 장기간의 무중력 상태,")
    df.loc[1236, 'dialogue'] = df.loc[1236, 'dialogue'].replace("#고객님, 크루즈 컨트롤에 대해", "#Person2#: 고객님, 크루즈 컨트롤에 대해")
    df.loc[1250, 'dialogue'] = df.loc[1250, 'dialogue'].replace("#여기 있습니다. 스티븐스", "#Person1#: 여기 있습니다. 스티븐스")
    df.loc[1266, 'dialogue'] = df.loc[1266, 'dialogue'].replace("#고객님, 저희는 고객이 화나거나", "#Person2#: 고객님, 저희는 고객이 화나거나")
    df.loc[1278, 'dialogue'] = df.loc[1278, 'dialogue'].replace("#고객님, 죄송합니다만 계산대에서", "#Person2#: 고객님, 죄송합니다만 계산대에서")
    df.loc[1281, 'dialogue'] = df.loc[1281, 'dialogue'].replace("#잠깐만요, 버전 7 요구 사항을", "#Person1#: 잠깐만요, 버전 7 요구 사항을")
    df.loc[1283, 'dialogue'] = df.loc[1283, 'dialogue'].replace("#어디 보자. 네, 그런 방이 두 개", "#Person1#: 어디 보자. 네, 그런 방이 두 개")
    df.loc[1301, 'dialogue'] = df.loc[1301, 'dialogue'].replace("#샐러드용 드레싱은 세 가지", "#Person1#: 샐러드용 드레싱은 세 가지")
    df.loc[1302, 'dialogue'] = df.loc[1302, 'dialogue'].replace("#페리에와 짐 빔 세 병씩", "#Person1#: 페리에와 짐 빔 세 병씩")
    df.loc[1306, 'dialogue'] = df.loc[1306, 'dialogue'].replace("#나 부엌에 있어", "#Person2#: 나 부엌에 있어")
    df.loc[1322, 'dialogue'] = df.loc[1322, 'dialogue'].replace("#여기서 만나서 반갑습니다.", "#Person1#: 여기서 만나서 반갑습니다.")
    df.loc[1424, 'dialogue'] = df.loc[1424, 'dialogue'].replace(" 방으로 가는 길은 어느", "#Person1#: 방으로 가는 길은 어느")
    df.loc[1497, 'dialogue'] = df.loc[1497, 'dialogue'].replace(" 복사 한 장당 비용은", "#Person1#: 복사 한 장당 비용은")
    df.loc[1547, 'dialogue'] = df.loc[1547, 'dialogue'].replace("#작은 걸로 주세요.", "#Person2#: 작은 걸로 주세요.")
    df.loc[1609, 'dialogue'] = df.loc[1609, 'dialogue'].replace("#여기 있습니다.", "#Person2#: 여기 있습니다.")
    df.loc[2240, 'dialogue'] = df.loc[2240, 'dialogue'].replace("사라가 왜 아직 안 왔지?", "#Person1#: 사라가 왜 아직 안 왔지?")
    df.loc[5812, 'dialogue'] = df.loc[5812, 'dialogue'].replace("공장에서의 모든 직원들이 거리에서", "#Person2#: 공장에서의 모든 직원들이 거리에서")
    df.loc[5812, 'dialogue'] = df.loc[5812, 'dialogue'].replace("오늘 2천 명의 직원 중 한 명도", "#Person1#: 오늘 2천 명의 직원 중 한 명도")

    # 두번 연속해서 말하는 데이터 수정
    df.loc[345, 'dialogue'] = df.loc[345, 'dialogue'].replace("#Person1#: 아니요. 입원할 필요는 없습니다.", "#Person2#: 아니요. 입원할 필요는 없습니다.")
    df.loc[484, 'dialogue'] = df.loc[484, 'dialogue'].replace("#Person1#: 인상적이네. 우리는 좋은 관계야.", "#Person1#: 인상적이네. \n#Person2#: 우리는 좋은 관계야.")
    df.drop(756, inplace=True)
    df.loc[872, 'dialogue'] = df.loc[872, 'dialogue'].replace("#Person1#: 중국은행에서 개설한", "#Person2#: 중국은행에서 개설한")
    df.drop(925, inplace=True)
    df.drop(982, inplace=True)
    df.loc[1033, 'dialogue'] = df.loc[1033, 'dialogue'].replace("#Person2#: 메리가 좋아할 만한 것에 대해 생각해봐야 해. 메리의 취미가 뭐야?", "#Person2#: 메리가 좋아할 만한 것에 대해 생각해봐야 해.\n#Person1#: 메리의 취미가 뭐야?")
    df.loc[1220, 'dialogue'] = df.loc[1220, 'dialogue'].replace("#Person2#: 음, 치아에 충치가 있고, 크라운도", "#Person1#: 음, 치아에 충치가 있고, 크라운도")
    df.drop(1294, inplace=True)
    df.loc[1419, 'dialogue'] = df.loc[1419, 'dialogue'].replace("때문이에요.\n#Person1#: 어떠세요?", "때문이에요. 어떠세요?")
    df.loc[1440, 'dialogue'] = df.loc[1440, 'dialogue'].replace("네. 방을 예약하고 싶습니다. 언제 예약하시겠습니까? ", "네. 방을 예약하고 싶습니다.\n#Person1#: 언제 예약하시겠습니까?")
    df.loc[1475, 'dialogue'] = df.loc[1475, 'dialogue'].replace("아르바이트가 필요해요.\n#Person2#: 그거 좋겠네요.\n", "아르바이트가 필요해요.\n")
    df.drop(1791, inplace=True)
    df.loc[1899, 'dialogue'] = df.loc[1899, 'dialogue'].replace("#Person1#: 한번 봐볼까. 이게 뭐야?", "#Person2#: 한번 봐볼까. 이게 뭐야?")
    df.loc[2240, 'dialogue'] = df.loc[2240, 'dialogue'].replace("다른 셔츠들과 함께 옷장에 있어. \n#Person1#: 사라가 왜 아직 안 왔지?", "다른 셔츠들과 함께 옷장에 있어. 사라가 왜 아직 안 왔지?")
    df.loc[3628, 'dialogue'] = df.loc[3628, 'dialogue'].replace("된 게 아니라고요. 당신의 전면 범퍼도 마찬가지로 망가져 있잖아요.", "된 게 아니라고요.\n#Person1#: 당신의 전면 범퍼도 마찬가지로 망가져 있잖아요.")
    df.loc[5441, 'dialogue'] = df.loc[5441, 'dialogue'].replace("#Person1#: 문제 없어!\n#Person1#: 안녕, 내가 왔어!", "#Person1#: 문제 없어! 안녕, 내가 왔어!")
    df.loc[6759, 'dialogue'] = df.loc[6759, 'dialogue'].replace("제 엄마는 코를 긁고요. \n#Person2#: 아, 눈-분명해요.", "제 엄마는 코를 긁고요. 아, 눈-분명해요.")
    df.drop(6799, inplace=True)
    df.loc[8645, 'dialogue'] = df.loc[8645, 'dialogue'].replace("#Person2#: 저는 서프라이즈 다운타운이라는", "#Person1#: 저는 서프라이즈 다운타운이라는")

    sample_list = df.loc[9898, 'dialogue_list']
    for i in range(1, len(sample_list)):
        if i % 2 != 0:
            sample_list[i] = sample_list[i].replace("#Person1#:", "#Person2#:")
        else:
            sample_list[i] = sample_list[i].replace("#Person2#:", "#Person1#:")

    df.loc[9898, 'dialogue'] = "\n".join(sample_list)

    df.loc[11578, 'dialogue'] = df.loc[11578, 'dialogue'].replace("그런데 쉽지 않았어요.\n#Person1#: 여기 보시겠어요?", "그런데 쉽지 않았어요. 여기 보시겠어요?")
    df['dialogue_list'] = df['dialogue'].str.split("\n")
    df.reset_index(drop=True, inplace=True)
    # df['dialogue'] = df['dialogue'].apply(resize_tokens)

    return df

In [ ]:
# gpt로 합성한 데이터 전처리
def gpt_train_process(df):
    train_df = pd.read_csv("../data/train.csv")
    df = train_df.merge(df, how='left', on='fname')

    df['summary_x_len'] = df['summary_x'].str.len()
    df['summary_y_len'] = df['summary_y'].str.len()

    # 원본 요약문과 gpt로 합성한 요약문의 길이의 차이가 너무 클 경우 제거
    df['summary_len_diff'] = abs(df['summary_x_len'] - df['summary_y_len'])
    df['summary_len_diff_rate'] = df['summary_len_diff'] / (df['summary_x_len'] + df['summary_y_len'])
    df = df[df['summary_len_diff_rate'] < 0.2].dropna()
    df = df[['fname', 'dialogue_x', 'summary_y']]
    df.columns = ['fname', 'dialogue', 'summary']
    
    return df

In [ ]:
# samsum 데이터 전처리
def samsum_process(df):
    df = df[['id', 'processed_dialogue', 'processed_summary']]
    df.columns = ['fname', 'dialogue', 'summary']
    df['dialogue_len'] = df['dialogue'].str.len()
    df['summary_len'] = df['summary'].str.len()
    
    df = df[df['summary_len'] > 10].reset_index(drop=True)  # 너무 요약문이 짧은 경우 제거
    return df

## 3 Prepare Dataset
- csv file 을 불러와서 전처리 후 encoder 와 decoder의 입력형태로 가공해줍니다.
- 가공된 데이터를 torch dataset class 로 구축하여 모델에 입력가능한 형태로 만듭니다.

In [ ]:
# 데이터 전처리를 위한 클래스로, 데이터셋을 데이터프레임으로 변환하고 인코더와 디코더의 입력을 생성합니다.
class Preprocess:
    def __init__(self,
            bos_token: str,
            eos_token: str,
        ) -> None:

        self.bos_token = bos_token
        self.eos_token = eos_token

    @staticmethod
    # 필요한 컬럼만 선택후 T5 모델에 맞게 prefix를 붙여줍니다.
    def make_set_as_df(df, is_train = True):
        if is_train:
            train_df = df[['fname','dialogue','summary']][0:1000]
            train_df["dialogue"] = "summarize: "+ train_df["dialogue"]
            return train_df
        else:
            test_df = df[['fname','dialogue']]
            test_df["dialogue"] = "summarize: "+ test_df["dialogue"]
            return test_df

    def make_input(self, dataset, is_test = False):
        if is_test:
            encoder_input = dataset['dialogue']
            decoder_input = [self.bos_token] * len(dataset['dialogue'])
            return encoder_input.tolist(), list(decoder_input)
        else:
            encoder_input = dataset['dialogue']
            decoder_input = dataset['summary']
            return encoder_input.tolist(), decoder_input.tolist()


In [ ]:
# Train에 사용되는 Dataset 클래스를 정의합니다.
class DatasetForTrain(Dataset):
    def __init__(self, encoder_input, labels, len):
        self.encoder_input = encoder_input
        self.labels = labels
        self.len = len

    def __getitem__(self, idx):
        item = {key: val[idx].clone().detach() for key, val in self.encoder_input.items()} # item[input_ids], item[attention_mask]
        item['labels'] = self.labels[idx]
        return item

    def __len__(self):
        return self.len

# Validation에 사용되는 Dataset 클래스를 정의합니다.
class DatasetForVal(Dataset):
    def __init__(self, encoder_input, labels, len):
        self.encoder_input = encoder_input
        self.labels = labels
        self.len = len

    def __getitem__(self, idx):
        item = {key: val[idx].clone().detach() for key, val in self.encoder_input.items()} # item[input_ids], item[attention_mask]
        item['labels'] = self.labels[idx]
        return item

    def __len__(self):
        return self.len

# Test에 사용되는 Dataset 클래스를 정의합니다.
class DatasetForInference(Dataset):
    def __init__(self, encoder_input, test_id, len):
        self.encoder_input = encoder_input
        self.test_id = test_id
        self.len = len

    def __getitem__(self, idx):
        item = {key: val[idx].clone().detach() for key, val in self.encoder_input.items()}
        item['ID'] = self.test_id[idx]
        return item

    def __len__(self):
        return self.len


In [ ]:
# 모든 전처리가 끝낙 최종적으로 모델에 입력될 최종적인 데이터를 출력
def prepare_train_dataset(config, preprocessor, data_path, tokenizer):
    train_file_path = os.path.join(data_path,'train.csv')
    gpt_file_path = os.path.join(data_path,'gpt_train.csv')
    samsum_file_path = os.path.join(data_path,'ko_samsum.csv')
    val_file_path = os.path.join(data_path,'dev.csv')

    # train, validation에 대해 각각 데이터프레임을 구축합니다.
    train_df = pd.read_csv(train_file_path)
    train_df = train_process(train_df)
    val_df = pd.read_csv(val_file_path)
    
    # gpt_df = pd.read_csv(gpt_file_path)
    # gpt_df = gpt_train_process(gpt_df)
    
    # samsum_df = pd.read_csv(samsum_file_path)
    # samsum_df = samsum_process(samsum_df)
    
    # train_df = pd.concat([train_df, gpt_df, samsum_df], axis=0)
    print(train_df.shape)
    
    train_data = preprocessor.make_set_as_df(train_df)
    val_data = preprocessor.make_set_as_df(val_df)

    print('-'*150)
    print(f'train_data:\n {train_data["dialogue"][0]}')
    print(f'train_label:\n {train_data["summary"][0]}')

    print('-'*150)
    print(f'val_data:\n {val_data["dialogue"][0]}')
    print(f'val_label:\n {val_data["summary"][0]}')
    
    encoder_input_train , decoder_input_train = preprocessor.make_input(train_data)
    encoder_input_val , decoder_input_val = preprocessor.make_input(val_data)
    print('-'*10, 'Load data complete', '-'*10,)
    
    tokenized_encoder_inputs = tokenizer(encoder_input_train, return_tensors="pt", padding=True,
                        add_special_tokens=True, truncation=True, max_length=config['tokenizer']['encoder_max_len'], return_token_type_ids=False)
    tokenized_decoder_inputs = tokenizer(decoder_input_train, return_tensors="pt", padding=True,
                        add_special_tokens=True, truncation=True, max_length=config['tokenizer']['decoder_max_len'], return_token_type_ids=False)
    
    train_inputs_dataset = DatasetForTrain(tokenized_encoder_inputs, tokenized_decoder_inputs['input_ids'], len(encoder_input_train))
    
    val_tokenized_encoder_inputs = tokenizer(encoder_input_val, return_tensors="pt", padding=True,
                        add_special_tokens=True, truncation=True, max_length=config['tokenizer']['encoder_max_len'], return_token_type_ids=False)
    val_tokenized_decoder_inputs = tokenizer(decoder_input_val, return_tensors="pt", padding=True,
                        add_special_tokens=True, truncation=True, max_length=config['tokenizer']['decoder_max_len'], return_token_type_ids=False)

    val_inputs_dataset = DatasetForVal(val_tokenized_encoder_inputs, val_tokenized_decoder_inputs['input_ids'], len(encoder_input_val))

    print('-'*10, 'Make dataset complete', '-'*10,)
    return train_inputs_dataset, val_inputs_dataset

## 4 Define Trainer & Trainingargs 

In [ ]:
# 학습 중 기록할 평가지표 정의
def compute_metrics(config,tokenizer,pred):
    rouge = Rouge()
    
    predictions = pred.predictions
    labels = pred.label_ids

    predictions[predictions == -100] = tokenizer.pad_token_id
    labels[labels == -100] = tokenizer.pad_token_id

    decoded_preds = tokenizer.batch_decode(predictions, clean_up_tokenization_spaces=True)
    labels = tokenizer.batch_decode(labels, clean_up_tokenization_spaces=True)
    
    # 정확한 평가를 위해 미리 정의된 불필요한 생성토큰들을 제거
    replaced_predictions = decoded_preds.copy()
    replaced_labels = labels.copy()
    remove_tokens = config['inference']['remove_tokens']
    for token in remove_tokens:
        replaced_predictions = [sentence.replace(token," ") for sentence in replaced_predictions]
        replaced_labels = [sentence.replace(token," ") for sentence in replaced_labels]
        
    print('-'*150)
    print(f"PRED1: {replaced_predictions[0]}")
    print(f"GOLD1: {replaced_labels[0]}")
    print('-'*150)
    print(f"PRED2: {replaced_predictions[1]}")
    print(f"GOLD2: {replaced_labels[1]}")
    print('-'*150)
    print(f"PRED3: {replaced_predictions[2]}")
    print(f"GOLD3: {replaced_labels[2]}")
    
    # 더욱 정확한 평가를 위해 형태소 단위로 나누기
    # mecab = Mecab()
    # replaced_predictions = list(map(lambda x: " ".join(mecab.morphs(x)), replaced_predictions))
    # replaced_labels = list(map(lambda x: " ".join(mecab.morphs(x)), replaced_labels))
    
    for i, sequence in enumerate(replaced_predictions):
        if sequence == "":
            replaced_predictions[i] = "언노운"

    # 최종적인 ROUGE 점수 계산
    results = rouge.get_scores(replaced_predictions, replaced_labels,avg=True)
    final_score = (results['rouge-1']['f'] + results['rouge-2']['f'] + results['rouge-l']['f']) / 3

    result = {key: value["f"] for key, value in results.items()}
    result['Final Score'] = final_score
    return result

In [ ]:
# trainer 매개변수 및 클래스 정의
def load_trainer_for_train(config,generate_model,tokenizer,train_inputs_dataset,val_inputs_dataset):
    print('-'*10, 'Make training arguments', '-'*10,)
    # set training args
    global training_args
    training_args = Seq2SeqTrainingArguments(
                output_dir=config['general']['output_dir'], # model output directory
                overwrite_output_dir=config['training']['overwrite_output_dir'],
                num_train_epochs=config['training']['num_train_epochs'],  # total number of training epochs
                learning_rate=config['training']['learning_rate'], # learning_rate
                per_device_train_batch_size=config['training']['per_device_train_batch_size'], # batch size per device during training
                per_device_eval_batch_size=config['training']['per_device_eval_batch_size'],# batch size for evaluation
                warmup_ratio=config['training']['warmup_ratio'],  # number of warmup steps for learning rate scheduler
                weight_decay=config['training']['weight_decay'],  # strength of weight decay
                lr_scheduler_type=config['training']['lr_scheduler_type'],
                optim =config['training']['optim'],
                gradient_accumulation_steps=config['training']['gradient_accumulation_steps'],
                evaluation_strategy=config['training']['evaluation_strategy'], # evaluation strategy to adopt during training
                save_strategy =config['training']['save_strategy'],
                save_total_limit=config['training']['save_total_limit'], # number of total save model.
                fp16=config['training']['fp16'],
                fp16_full_eval=config['training']['fp16_full_eval'],
                load_best_model_at_end=config['training']['load_best_model_at_end'], # 최종적으로 가장 높은 점수 저장
                seed=config['training']['seed'],
                logging_dir=config['training']['logging_dir'], # directory for storing logs
                logging_strategy=config['training']['logging_strategy'],
                predict_with_generate=config['training']['predict_with_generate'], #To use BLEU or ROUGE score
                generation_max_length=config['training']['generation_max_length'],
                do_train=config['training']['do_train'],
                do_eval=config['training']['do_eval'],
                report_to=config['training']['report_to'] # (선택) wandb를 사용할 때 설정
            )

    # (선택) wandb 초기화
    wandb.init(
        entity=config['wandb']['entity'],
        project=config['wandb']['project'],
        name=config['wandb']['name'],
    )

    # (선택) 모델 checkpoint를 wandb에 저장하도록 환경 변수를 설정
    os.environ["WANDB_LOG_MODEL"]="true"
    os.environ["WANDB_WATCH"]="false"

    # EarlyStopping
    MyCallback = EarlyStoppingCallback(
        early_stopping_patience=config['training']['early_stopping_patience'],
        early_stopping_threshold=config['training']['early_stopping_threshold']
    )
    print('-'*10, 'Make training arguments complete', '-'*10,)
    print('-'*10, 'Make trainer', '-'*10,)

    # Trainer 클래스를 정의
    trainer = Seq2SeqTrainer(
        model=generate_model, # 사용자가 사전 학습하기 위해 사용할 모델을 입력합니다.
        args=training_args,
        train_dataset=train_inputs_dataset,
        eval_dataset=val_inputs_dataset,
        compute_metrics = lambda pred: compute_metrics(config,tokenizer, pred),
        callbacks = [MyCallback]
    )
    print('-'*10, 'Make trainer complete', '-'*10,)
    return trainer

In [ ]:
# tokenizer와 모델 불러오기
def load_tokenizer_and_model_for_train(config,device):
    print('-'*10, 'Load tokenizer & model', '-'*10,)
    print('-'*10, f'Model Name : {config["general"]["model_name"]}', '-'*10,)
    
    model_name = config['general']['model_name']
    custom_bart_config = {'num_beams': 4}
    t5_config = T5Config().from_pretrained(model_name)
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    generate_model = AutoModelForSeq2SeqLM.from_pretrained(config['general']['model_name'], config=t5_config)

    special_tokens_dict={'additional_special_tokens':config['tokenizer']['special_tokens']}
    tokenizer.add_special_tokens(special_tokens_dict)

    generate_model.resize_token_embeddings(len(tokenizer)) # 사전에 special token을 추가했으므로 재구성 해줍니다.
    generate_model.to(device)
    print(generate_model.config)

    print('-'*10, 'Load tokenizer & model complete', '-'*10,)
    return generate_model , tokenizer

## 5 모델 학습하기

- 앞에서 구축한 클래스 및 함수를 활용하여 학습 진행합니다.

In [ ]:
def main(config):
    device = torch.device('cuda:0' if torch.cuda.is_available()  else 'cpu')
    print('-'*10, f'device : {device}', '-'*10,)
    print(torch.__version__)
    
    # 시드 고정
    pl.seed_everything(seed=42, workers=False)

    # 모델 및 토크나이저 로드
    generate_model , tokenizer = load_tokenizer_and_model_for_train(config,device)
    print('-'*10,"tokenizer special tokens : ",tokenizer.special_tokens_map,'-'*10)

    # 데이터셋 준비
    preprocessor = Preprocess(config['tokenizer']['bos_token'], config['tokenizer']['eos_token']) # decoder_start_token: str, eos_token: str
    data_path = config['general']['data_path']
    
    train_inputs_dataset, val_inputs_dataset = prepare_train_dataset(config,preprocessor, data_path, tokenizer)
    
    # Trainer 클래스 로드
    trainer = load_trainer_for_train(config, generate_model,tokenizer,train_inputs_dataset,val_inputs_dataset)
    trainer.train()   # 모델 학습을 시작합니다.
    
    generate_model.save_pretrained("../model/bestmodel/")

    # (선택) wandb 종료
    wandb.finish()

In [ ]:
if __name__ == "__main__":
    main(loaded_config)

Seed set to 42


---------- device : cuda:0 ----------
2.1.2+cu118
---------- Load tokenizer & model ----------
---------- Model Name : eenzeenee/t5-base-korean-summarization ----------
T5Config {
  "_name_or_path": "eenzeenee/t5-base-korean-summarization",
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "classifier_dropout": 0.0,
  "d_ff": 2048,
  "d_kv": 64,
  "d_model": 768,
  "decoder_start_token_id": 0,
  "dense_act_fn": "gelu_new",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "gated-gelu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": true,
  "layer_norm_epsilon": 1e-06,
  "max_length": 128,
  "model_type": "t5",
  "num_decoder_layers": 12,
  "num_heads": 12,
  "num_layers": 12,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,
  "tie_word_embeddings": false,
  "torch_dtype": "float32",
  "transformers_version": "4.35.2",
  "use_cache": true,
  "vocab_size"

/tmp/ipykernel_902491/3690234399.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df["dialogue"] = "summarize: "+ train_df["dialogue"]
/tmp/ipykernel_902491/3690234399.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df["dialogue"] = "summarize: "+ train_df["dialogue"]


---------- Make dataset complete ----------
---------- Make training arguments ----------


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: dudcjs2779. Use `wandb login --relogin` to force relogin


...

---------- Make training arguments complete ----------
---------- Make trainer ----------
---------- Make trainer complete ----------


Epoch,Training Loss,Validation Loss,Rouge-1,Rouge-2,Rouge-l,Final score
1,3.572800,0.549321,0.236181,0.101147,0.251974,0.196434
2,0.505800,0.391079,0.503167,0.312450,0.454233,0.423283
3,0.427800,0.372069,0.509981,0.324326,0.459517,0.431275
4,0.394900,0.363155,0.530329,0.346910,0.474607,0.450615
5,0.370700,0.361740,0.528458,0.345054,0.472375,0.448629
6,0.350000,0.358526,0.528427,0.346786,0.470502,0.448572
7,0.332600,0.359302,0.530108,0.351052,0.477342,0.452834
8,0.317300,0.358019,0.535632,0.355153,0.478902,0.456562
9,0.304300,0.360800,0.535356,0.355512,0.478389,0.456419
10,0.292100,0.365446,0.533372,0.352662,0.478790,0.454941


------------------------------------------------------------------------------------------------------------------------------------------------------
PRED1:  숨쉬기가 힘들어요. 그는 천식에 대한 검사를 받게 할 것입니다.                                                                                                     
GOLD1: #Person2#는 숨쉬기에 어려움을 겪는다. 의사는 #Person1#에게 이에 대해 묻고, #Person2#를 폐 전문의에게 보낼 예정이다.                                                                                        
------------------------------------------------------------------------------------------------------------------------------------------------------
PRED2:  지미는 오늘 운동을 위해 헬스장에 가기로 결정했다. 지미는 오늘 두 날을 바꾸는 것을 제안한다.                                                                                        
GOLD2: #Person1#은 지미에게 운동하러 가자고 제안하고 팔과 배를 운동하도록 설득한다.                                                                                                      
-----------------------------------------------------------------------

There were missing keys in the checkpoint model loaded: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight'].
Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


eval/Final Score,▁▇▇████████
eval/loss,█▂▂▁▁▁▁▁▁▁▁
eval/rouge-1,▁▇▇████████
eval/rouge-2,▁▇▇████████
eval/rouge-l,▁▇▇████████
eval/runtime,▃▃▅▂▆▅▅▃▁█▇
eval/samples_per_second,▆▆▄▇▃▄▄▅█▁▂
eval/steps_per_second,▆▆▄▇▃▄▄▅█▁▂
train/epoch,▁▁▂▂▂▂▃▃▄▄▅▅▅▅▆▆▇▇▇▇███
train/global_step,▁▁▂▂▂▂▃▃▄▄▅▅▅▅▆▆▇▇▇▇███
train/learning_rate,▁███▇▆▅▅▃▂▁


In [ ]:
gwah

## 6 모델 추론하기

### 6.1 Test Inference

In [ ]:
# 이곳에 내가 사용할 wandb config 설정
loaded_config['inference']['ckt_path'] = "../model/bestmodel"
# loaded_config['inference']['ckt_path'] = "../model/checkpoint-12456"
# loaded_config['inference']['num_beams'] = 6
# loaded_config['inference']['no_repeat_ngram_size'] = 6
# loaded_config['inference']['generate_max_length'] = 128
# loaded_config['inference']

In [ ]:
# 모든 전처리가 끝낙 최종적으로 모델에 입력될 최종적인 데이터를 출력
def prepare_test_dataset(config,preprocessor, tokenizer, is_valid):

    if is_valid:
        test_file_path = os.path.join(config['general']['data_path'],'dev.csv')
    else:
        test_file_path = os.path.join(config['general']['data_path'],'test.csv')

    test_df = pd.read_csv(test_file_path)
    test_data = preprocessor.make_set_as_df(test_df, is_train=is_valid)
    test_id = test_data['fname']

    print('-'*150)
    print(f'test_data:\n{test_data["dialogue"][0]}')
    print('-'*150)

    encoder_input_test , decoder_input_test = preprocessor.make_input(test_data,is_test=True)
    print('-'*10, 'Load data complete', '-'*10,)

    test_tokenized_encoder_inputs = tokenizer(encoder_input_test, return_tensors="pt", padding=True,
                    add_special_tokens=True, truncation=True, max_length=config['tokenizer']['encoder_max_len'], return_token_type_ids=False,)
    test_tokenized_decoder_inputs = tokenizer(decoder_input_test, return_tensors="pt", padding=True,
                    add_special_tokens=True, truncation=True, max_length=config['tokenizer']['decoder_max_len'], return_token_type_ids=False,)

    test_encoder_inputs_dataset = DatasetForInference(test_tokenized_encoder_inputs, test_id, len(encoder_input_test))
    print('-'*10, 'Make dataset complete', '-'*10,)

    return test_data, test_encoder_inputs_dataset

In [ ]:
# 추론시 사용할 모델과 토크나이저 로드
def load_tokenizer_and_model_for_test(config, device):
    print('-'*10, 'Load tokenizer & model', '-'*10,)

    model_name = config['general']['model_name']
    ckt_path = config['inference']['ckt_path']
    print('-'*10, f'Model Name : {model_name}', '-'*10,)
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    special_tokens_dict = {'additional_special_tokens': config['tokenizer']['special_tokens']}
    tokenizer.add_special_tokens(special_tokens_dict)

    generate_model = AutoModelForSeq2SeqLM.from_pretrained(ckt_path)
    generate_model.resize_token_embeddings(len(tokenizer))
    generate_model.to(device)
    print('-'*10, 'Load tokenizer & model complete', '-'*10,)

    return generate_model , tokenizer

In [ ]:
# 추론 output 파일 생성
def inference(config):
    device = torch.device('cuda:0' if torch.cuda.is_available()  else 'cpu')
    print('-'*10, f'device : {device}', '-'*10,)
    print(torch.__version__)

    generate_model , tokenizer = load_tokenizer_and_model_for_test(config,device)

    data_path = config['general']['data_path']
    preprocessor = Preprocess(config['tokenizer']['bos_token'], config['tokenizer']['eos_token'])

    test_data, test_encoder_inputs_dataset = prepare_test_dataset(config,preprocessor, tokenizer, False)
    dataloader = DataLoader(test_encoder_inputs_dataset, batch_size=config['inference']['batch_size'])

    summary = []
    text_ids = []
    generate_model.eval()
    with torch.no_grad():
        for item in tqdm(dataloader):
            text_ids.extend(item['ID'])
            generated_ids = generate_model.generate(input_ids=item['input_ids'].to('cuda:0'),
                            no_repeat_ngram_size=config['inference']['no_repeat_ngram_size'],
                            early_stopping=config['inference']['early_stopping'],
                            max_length=config['inference']['generate_max_length'],
                            num_beams=config['inference']['num_beams'],
                        )
            for ids in generated_ids:
                result = tokenizer.decode(ids)
                summary.append(result)

    # 정확한 평가를 위하여 노이즈에 해당되는 스페셜 토큰을 제거합니다.
    remove_tokens = config['inference']['remove_tokens']
    preprocessed_summary = summary.copy()
    for token in remove_tokens:
        preprocessed_summary = [sentence.replace(token," ") for sentence in preprocessed_summary]

    output = pd.DataFrame(
        {
            "fname": test_data['fname'],
            "summary" : preprocessed_summary,
        }
    )
    result_path = config['inference']['result_path']
    if not os.path.exists(result_path):
        os.makedirs(result_path)
    output.to_csv(os.path.join(result_path, "output.csv"), index=False)

    return output

In [ ]:
# 학습된 모델의 test를 진행합니다.
if __name__ == "__main__":
    output = inference(loaded_config)

---------- device : cuda:0 ----------
2.1.2+cu118
---------- Load tokenizer & model ----------
---------- Model Name : eenzeenee/t5-base-korean-summarization ----------
---------- Load tokenizer & model complete ----------
------------------------------------------------------------------------------------------------------------------------------------------------------
test_data:
summarize: #Person1#: 더슨 씨, 받아쓰기 좀 해주세요. 
#Person2#: 네, 실장님...
#Person1#: 이것은 오늘 오후까지 모든 직원에게 내부 메모로 전달되어야 합니다. 준비되셨나요?
#Person2#: 네, 실장님. 시작하셔도 됩니다.
#Person1#: 모든 직원들에게 주의하라... 즉시 효력을 발휘하여, 모든 사무실 통신은 이메일 통신과 공식 메모로 제한됩니다. 근무 시간 동안 직원들이 즉시 메시지 프로그램을 사용하는 것은 엄격히 금지됩니다.
#Person2#: 실장님, 이것은 내부 통신에만 적용되는 건가요? 아니면 외부 통신에도 제한이 되는 건가요?
#Person1#: 이것은 모든 통신에 적용되어야 합니다, 이 사무실 내의 직원들 사이뿐만 아니라 외부 통신에도 마찬가지입니다.
#Person2#: 하지만 실장님, 많은 직원들이 고객과 소통하기 위해 즉시 메시지를 사용하고 있습니다.
#Person1#: 그들은 그들의 의사소통 방법을 바꾸어야만 합니다. 이 사무실에서 누구도 즉시 메시지를 사용하지 않기를 원합니다. 너무 많은 시간을 낭비하게 됩니다! 이제, 메모를 계속해주세요. 우리가 어디까지 했나요?
#Person2#: 이것은 내부와 외부 통신에 적용

100%|██████████| 63/63 [02:24<00:00,  2.29s/it]


In [ ]:
output

,fname,summary
0,test_0,더슨 씨는 직원에게 내부 메모가 이메일 통신과 공식 메모로 제한된다고 말합니다....
1,test_1,#Person1#과 #Person2#는 출퇴근 시간에 항상 교통이 많이 밀리는 ...
2,test_2,케이트는 마샤와 히어로가 이혼하려고 한다고 #Person1#에게 말한다. 케이트...
3,test_3,#Person1#은 브라이언의 생일을 축하하기 위해 파티를 즐기고 있다. ...
4,test_4,#Person1#과 #Person2#는 올림픽 스타디움에 대해 이야기하고 있다....
...,...,...
494,test_495,찰리는 잭에게 자신의 캐릭터를 만드는 비디오 게임을 요청한다. 잭은 그것이 흥미...
495,test_496,#Person2#는 #Person1#에게 컨트리 음악 레코드를 더 많이 사고 있...
496,test_497,앨리스는 세탁기라는 기계 안에 비누를 넣어야 한다고 #Person1#에게 말한다...
497,test_498,스티브가 매튜에게 계약 갱신을 위해 살 곳을 찾고 있다고 말한다. 스티브는 그녀...


### 6.2 Valid Inference

In [ ]:
# valid 추론
device = torch.device('cuda:0' if torch.cuda.is_available()  else 'cpu')
print('-'*10, f'device : {device}', '-'*10,)
print(torch.__version__)
print(loaded_config['inference'])

generate_model , tokenizer = load_tokenizer_and_model_for_test(loaded_config, device)

data_path = loaded_config['general']['data_path']
preprocessor = Preprocess(loaded_config['tokenizer']['bos_token'], loaded_config['tokenizer']['eos_token'])

val_data, val_encoder_inputs_dataset = prepare_test_dataset(loaded_config, preprocessor, tokenizer, True)
dataloader = DataLoader(val_encoder_inputs_dataset, batch_size=loaded_config['inference']['batch_size'])

summary = []
text_ids = []
generate_model.eval()
with torch.no_grad():
    for item in tqdm(dataloader):
        text_ids.extend(item['ID'])
        generated_ids = generate_model.generate(input_ids=item['input_ids'].to('cuda:0'),
                        no_repeat_ngram_size=loaded_config['inference']['no_repeat_ngram_size'],
                        early_stopping=loaded_config['inference']['early_stopping'],
                        max_length=loaded_config['inference']['generate_max_length'],
                        num_beams=loaded_config['inference']['num_beams'],
                    )
        for ids in generated_ids:
            result = tokenizer.decode(ids)
            summary.append(result)

# 정확한 평가를 위해 미리 정의된 불필요한 생성토큰들을 제거
remove_tokens = loaded_config['inference']['remove_tokens']
preprocessed_summary = summary.copy()
for token in remove_tokens:
    preprocessed_summary = [sentence.replace(token,"") for sentence in preprocessed_summary]

val_data['pred'] = preprocessed_summary

# 더욱 정확한 평가를 위해 형태소 단위로 나누기
mecab = Mecab()
val_data['pred_morphs'] = val_data['pred'].apply(lambda x: " ".join(mecab.morphs(x)))
val_data['summary_morphs'] = val_data['summary'].apply(lambda x: " ".join(mecab.morphs(x)))

# ROUGE 점수 계산
rouge = Rouge()
rouge_socres = rouge.get_scores(val_data['pred_morphs'], val_data['summary_morphs'], avg=False)

rouge_1 = []
rouge_2 = []
rouge_L = []
for i in range(len(rouge_socres)):
    rouge_1.append(rouge_socres[i]['rouge-1']['f'])
    rouge_2.append(rouge_socres[i]['rouge-2']['f'])
    rouge_L.append(rouge_socres[i]['rouge-l']['f'])
    
final_score = (np.array(rouge_1).mean() + np.array(rouge_2).mean() + np.array(rouge_L).mean()) / 3

# EDA를 위해 데이터별로 rouge 스코어 계산
val_data['rouge_1'] = rouge_1
val_data['rouge_2'] = rouge_2
val_data['rouge_L'] = rouge_L

print("--"*30)
print(f"Rouge-1: {np.array(rouge_1).mean():.4f}, Rouge-2: {np.array(rouge_2).mean():.4f}, Rouge-l: {np.array(rouge_L).mean():.4f},")
print(f"Final Score: {final_score}")

---------- device : cuda:0 ----------
2.1.2+cu118
{'batch_size': 8, 'ckt_path': '../model/bestmodel', 'early_stopping': True, 'generate_max_length': 128, 'no_repeat_ngram_size': 6, 'num_beams': 6, 'remove_tokens': ['<usr>', 'None', '</s>', '<pad>'], 'result_path': './prediction/'}
---------- Load tokenizer & model ----------
---------- Model Name : eenzeenee/t5-base-korean-summarization ----------


/tmp/ipykernel_925410/3690234399.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df["dialogue"] = "summarize: "+ train_df["dialogue"]


---------- Load tokenizer & model complete ----------
------------------------------------------------------------------------------------------------------------------------------------------------------
test_data:
summarize: #Person1#: 안녕하세요, 오늘 하루 어떠셨어요? 
#Person2#: 요즘 숨쉬기가 좀 힘들어요.
#Person1#: 최근에 감기 같은 것에 걸리신 적이 있나요?
#Person2#: 아니요, 감기는 아니에요. 그냥 숨을 쉴 때마다 가슴이 무겁게 느껴져요.
#Person1#: 알고 있는 알레르기가 있나요?
#Person2#: 아니요, 알고 있는 알레르기는 없어요.
#Person1#: 이런 증상이 항상 나타나나요, 아니면 활동할 때 주로 나타나나요?
#Person2#: 운동을 할 때 많이 나타나요.
#Person1#: 저는 당신을 폐 전문의에게 보내서 천식에 대한 검사를 받게 할 거예요.
#Person2#: 도와주셔서 감사합니다, 의사 선생님.
------------------------------------------------------------------------------------------------------------------------------------------------------
---------- Load data complete ----------
---------- Make dataset complete ----------


100%|██████████| 63/63 [02:17<00:00,  2.18s/it]


------------------------------------------------------------
Rouge-1: 0.5359, Rouge-2: 0.3566, Rouge-l: 0.4765,
Final Score: 0.4563331316849195


In [ ]:
# 예측을 못한, 잘한 케이스 비교
bad_pred = val_data.sort_values(by='rouge_1').head(100)
good_pred = val_data.sort_values(by='rouge_1', ascending=False).head(100)

In [ ]:
print_data(bad_pred, print_random=True)

Index([343, 104, 210, 365, 237], dtype='int64')
[dev_343]
[Dialogue]
summarize: #Person1#: 아, 제가 논문 작업을 시작한 이후로 컴퓨터 화면에 갑자기 나타나는 광고가 벌써 네 번째네요.
#Person2#: 그런 광고를 막아주는 앱을 사면 됩니다.
#Person1#: 논문을 쓰기 위해 고가의 소프트웨어를 사는 건 감당이 안 돼요.
#Person2#: 그렇게 비싼 것도 아니에요. 한 달에 1달러밖에 안 해요.
#Person1#: 그러면 일년에 12달러네요.
#Person2#: 그 논문을 쓰는 데 일년이나 걸릴 건가요?
#Person1#: 아니, 해리엇. 3주 안에 끝낼 거예요. 하지만 졸업하기 전까지 4년 동안 이 컴퓨터로 학교 과제를 할 거거든요.
#Person2#: 그럼 가치가 있을 거라고 생각해요, 존. 그리고 30달러를 지불하면 월 비용을 내지 않아도 돼요.
#Person1#: 4년 동안 30달러요?
#Person2#: 아니요, 한 번 지불하면 영원히 사용할 수 있어요.
[Summary]
존은 컴퓨터 화면에 나타난 광고에 집중이 흐트러집니다. 해리엇은 광고를 막기 위해 합리적인 가격의 앱을 사는 것을 존에게 추천합니다.
[Prediction]
해리엇은 존에게 컴퓨터 광고를 막아주는 앱을 사는 것을 제안한다. 존은 그것이 가치가 있다고 생각하며, 한 번 지불하면 영원히 사용할 수 있다고 말한다.
rouge_1: 0.4211, rouge_2: 0.2432, rouge_L: 0.3810
[dev_104]
[Dialogue]
summarize: #Person1#: 정말 운이 좋았어요. 마지막 2인석을 얻었는데---예약도 안 했었어요! 우리 뒤에 긴 줄 보셨나요?
#Person2#: 응, 오래 기다리지 않아서 다행이야. 배가 너무 고파!
#Person1#: 메뉴를 한 번 봐서 주문해봅시다. 나눠 먹을 에피타이저를 고를래요?
#Person2#: 삼모사와 파파돔 중에 어떤 게 더 좋아?
#Perso

In [ ]:
print_data(good_pred, print_random=True)

[dev_67]
[Dialogue]
#Person1#: 왕푸징 그랜드 호텔입니다. 어떻게 도와드릴까요?
#Person2#: 다음 토요일과 일요일 밤에 예약 가능한 방이 있나요?
#Person1#: 잠시만 기다려주시겠어요? 해당 날짜에 대한 방의 예약 가능 여부를 확인해보겠습니다. . . 네, 다음 주말에 몇 개의 방이 비어 있습니다. 저희는 일본식, 로마식, 프랑스식, 프레지던트 스위트 스타일의 더블룸, 스위트룸, 디럭스 스위트룸을 보유하고 있습니다. 어떤 스타일을 선호하시나요?
#Person2#: 더블룸으로 부탁드립니다.
#Person1#: 알겠습니다. 고객님의 성함을 알려주시겠어요?
#Person2#: 제 이름은 모니카 셀러입니다.
#Person1#: 알겠습니다, 모니카 님. 다음 토요일과 일요일 밤에 더블룸을 예약해드렸습니다. 그때 뵙겠습니다!
#Person2#: 감사합니다.
[Summary]
모니카가 다음 토요일과 일요일 밤에 방을 예약하기 위해 리셉션에 전화를 걸고, #Person1#이 그녀를 도와줍니다.
[Prediction]
 모니카 셀러는 다음 토요일과 일요일 밤에 더블룸을 예약하기 위해 왕푸징 그랜드 호텔에 전화를 합니다. 이 호텔에는 몇 개의 스위트룸이 있습니다. 
rouge_1: 0.5000, rouge_2: 0.3548, rouge_L: 0.5357
[dev_362]
[Dialogue]
#Person1#: 괜찮아, 이단? 평소처럼 밝아 보이지 않아.
#Person2#: 솔직히 말하자면, 에이바, 나는 방금 정말 안 좋은 하루를 보냈어.
#Person1#: 무슨 일이 있었어?
#Person2#: 먼저, 알람을 듣지 못하고 잠에서 깨어나니 일에 두 시간이나 늦었어.
#Person1#: 네 상사는 뭐라고 했어?
#Person2#: 그는 내가 다시 늦게 출근하면 나를 해고할 것이라고 말했어. 정말로 무서워!
#Person1#: 그게 끔찍하네. 회사에 지각한 게 처음이었어?
#Person2#: 그게 두 번째였어. 첫 번째는 차 사고가 났었거든.
#